In [1]:
import shutil 
import numpy as np
import os
import pyvista as pv
import sys
# sys.path.insert(0, '/home/pandu/Panresearch_local/PPP_Utility')
# from panpv import panmesh as pm
from os import listdir
import pdb
import torch_geometric
import torch
import mesh2graph_utility as mutl
from tqdm import tqdm
from torch_geometric.data import Data, Batch, Dataset

## data prepare

In [2]:
import re
import vtk
path = "../../data/cases/dataset1/"
file_name_list = os.listdir(path)
file_name_list = sorted(file_name_list, key=lambda x:float(re.findall("(\d+)",x)[0]))
N = len(file_name_list); print('total num of data:',N)

total num of data: 89


In [25]:
def vtk2vtp(invtkfile, outvtpfile, binary=False):
    """What it says on the label"""
    reader = vtk.vtkUnstructuredGridReader()
    reader.SetFileName(invtkfile)
    reader.Update()

    reader2 = vtk.vtkGeometryFilter()
    reader2.SetInputData(reader.GetOutput())
    reader2.Update
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(outvtpfile)
    if binary:
        writer.SetFileTypeToBinary()
    writer.SetInputConnection(reader2.GetOutputPort())
    writer.Update()

mesh_list = []
files = []
for i in range(N):
    invtkfile = '../../data/cases/dataset1/'+file_name_list[i]+'/Mapped_Blade_Surface.vtk'
    outvtpfile = '../../data/cases/dataset1/'+file_name_list[i]+'/Mapped_Blade_Surface.vtp'
    vtk2vtp(invtkfile, outvtpfile,binary=False)
    mesh_list.append(pv.read(outvtpfile))
    files.append(outvtpfile)

In [22]:
import vtk
from torch_geometric.utils import to_undirected

def cellArrayDivider(input_array):
    N = len(input_array)
    cursor = 0
    head_id = []
    segs = []
    
    while(cursor<N):
        head_id.append(cursor)
        segs.append(input_array[cursor+1:cursor+input_array[cursor]+1])
        cursor = cursor+input_array[cursor]+1
        # print(cursor)
    return segs

def readvtp(file_name):
    reader = vtk.vtkPolyDataReader()
    reader.SetFileName(file_name)
    reader.Update()

    return reader.GetOutput()

def vtk2GraphVertex(input_file, with_data = False, p_arrayid = 0, wss_arrayid = 1):
    if type(input_file) == str:
        # aorta_vtk = readvtp(input_file)
        rotorblade = pv.read(input_file)
    else:
        rotorblade = input_file
    segs = np.array(cellArrayDivider(rotorblade.faces))
    points = np.array(rotorblade.points,dtype = np.float32)
    nodal_normals = np.array(rotorblade.point_normals,dtype = np.float32)
    if with_data == True:
        nodal_pressure = rotorblade.point_data['MappedAbsoluteTotalPressure']
        nodal_wss = rotorblade.point_data['MappedTau']

    nodal_stmdist = rotorblade.point_data['MappedDerivedVelocityMagnitude']  
    nodal_features = np.array(nodal_normals, dtype = np.float32)
    transform = torch_geometric.transforms.FaceToEdge(remove_faces=False)
    if with_data == True:
        nodal_labels = np.hstack((np.array(nodal_pressure,dtype = np.float32)[:,np.newaxis], np.array(nodal_wss,dtype = np.float32)[:,np.newaxis])) # four dimensions
        readme = {'nodal_features':'normalx, normaly, normalz',
                # 'edge_features':'d_coordinatex, d_coordinatey, d_coordinatez, d_coordinatenorm',
                'nodal_pos':'corrdinatex, corrdinatey, corrdinatez',
                'nodal_lables':'pressure, wss',
                'nodal_norms':'normalx, normaly, normalz',
                'nodal_stmdist': 'stmdist', 'face':'triangles'}
        mesh_graph = Data(x = torch.tensor(nodal_features), #edge_index = torch.tensor(edge_connectivity.T),
                        # edge_attr = torch.tensor(edge_features), 
                        y = torch.tensor(nodal_labels), 
                        pos = torch.tensor(points), norm = torch.tensor(nodal_normals), stmdist = torch.tensor(np.array(nodal_stmdist,dtype = np.float32)[:,np.newaxis]),
                        face = torch.tensor(np.array(segs.T)))
    else:
        readme = {'nodal_features':'normalx, normaly, normalz',
                # 'edge_features':'d_coordinatex, d_coordinatey, d_coordinatez, d_coordinatenorm',
                'nodal_pos':'corrdinatex, corrdinatey, corrdinatez',
                'nodal_norms':'normalx, normaly, normalz',
                'nodal_stmdist': 'stmdist', 'face':'triangles'}
        mesh_graph = Data(x = torch.tensor(nodal_features),#edge_index = torch.tensor(edge_connectivity.T),
                        # edge_attr = torch.tensor(edge_features),
                        pos = torch.tensor(points), norm = torch.tensor(nodal_normals),stmdist = torch.tensor(np.array(nodal_stmdist,dtype = np.float32)[:,np.newaxis]),
                        face = torch.tensor(np.array(segs.T)))

    mesh_graph_transformed = transform(mesh_graph)
    return mesh_graph_transformed, readme

In [23]:
from torch_geometric.data import Dataset, Data
from feature_descriptors import *


class MyCustomDatasetVertex(Dataset):
    def __init__(self,root,filenames):
        self.filename = filenames
        super(MyCustomDatasetVertex, self).__init__(root)

    @property
    def raw_file_names(self):
        return self.filename
    
    @property
    def processed_file_names(self):

        return ['data_{:04d}.pt'.format(i) for i in range(4)]
    def download(self):
        pass

    def process(self):
        idx = 0
        for raw_path in self.raw_paths:
            # converting vtp to graph
            data,readme = vtk2GraphVertex(raw_path,with_data = True)
            # adding features 
            fd = FeatureDescriptors()
            data = fd(data)
            torch.save(data, os.path.join(self.processed_dir, 'data_{:04d}.pt'.format(idx)))
            idx += 1

    def __len__(self):
        return len(self.processed_file_names)

    def __getitem__(self, idx):
        data = torch.load(os.path.join(self.processed_dir, self.processed_file_names[idx]))
        return data

In [24]:
test_dataset = MyCustomDatasetVertex('../../data/vertex/')

Processing...


FileNotFoundError: File (/storage/pandu/PPP_Heart/data/vertex/raw/aorta0000.vtp) not found

In [16]:
pick_id = 0
demo,_ = vtk2GraphVertex(mesh_list[pick_id], with_data = True) # x is pos, y is pressure and tau

In [18]:
def normalize(data):
    max, min  = torch.max(data ,dim = 0)[0], torch.min(data ,dim = 0)[0]
    print('max, min', max,min)
    norm_data = (data-min)/(max-min)*2-1
    return norm_data, [max,min]
def denormalize(data, maxmin = None):
    max, min  = maxmin[0], maxmin[1]
    denorm_data = (data+1)/2*(max-min)+min
    return denorm_data

In [19]:
demo.ydenorm = demo.y
demo.y, maxmin = normalize(demo.y) ;print('normalized data',demo.y)

max, min tensor([127279.2500,    674.1190]) tensor([9.8850e+04, 1.2352e+00])
normalized data tensor([[ 0.6006, -0.9215],
        [ 0.5852, -0.8991],
        [ 0.5634, -0.6746],
        ...,
        [ 0.4209, -0.6052],
        [ 0.3443, -0.6269],
        [ 0.3816, -0.7038]])


In [21]:
# calculate the norm of edges 
xij = torch.stack([demo.pos[demo.edge_index[1,i]]-demo.pos[demo.edge_index[0,i]] for i in range(demo.edge_index.size()[1])])
# calculate the norm of edges 
xijabs = torch.stack([torch.norm(demo.pos[demo.edge_index[1,i]]-demo.pos[demo.edge_index[0,i]] ) for i in range(demo.edge_index.size()[1])])
demo.edge_attr = torch.cat((xij, xijabs.unsqueeze(1)),dim = 1); print('edge attr size',demo.edge_attr.size())
print('edge attr:',demo.edge_attr)

edge attr size torch.Size([60212, 4])
edge attr: tensor([[ 2.2189e-04,  1.4070e-04,  8.6654e-05,  2.7666e-04],
        [ 4.8425e-05,  2.3432e-04,  1.1091e-04,  2.6373e-04],
        [-2.3447e-04,  5.8639e-05, -6.7937e-05,  2.5105e-04],
        ...,
        [-5.1278e-05, -7.4301e-06,  3.1101e-04,  3.1530e-04],
        [-1.2711e-04,  1.2032e-04,  4.7548e-05,  1.8137e-04],
        [-7.6798e-05, -2.1863e-04,  1.2378e-04,  2.6272e-04]])


In [ ]:
class My_model(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=6, improved=False,
                 cached=False, bias=True, fine_marker_dict=None, GCNtype = 'MGN'):
        super().__init__()
        self.GCNtype = GCNtype
        channels = [in_channels]
        channels += [hidden_channels] * (num_layers - 1)
        channels.append(out_channels)

        convs = []
        for i in range(num_layers):
            if GCNtype == 'MGN':
                convs.append(MGNGraphNet(3, in_channels=channels[i], in_edge_channels=4, out_channels=channels[i+1],hidFeature=128,aggr='mean'))
            if GCNtype == 'GCN':
                convs.append(GCNConv(in_channels=channels[i], out_channels=channels[i+1]))
        self.convs = torch.nn.ModuleList(convs)

    def forward(self, data):
        tmp = [data.featr3[:, 0][:, [0, 0, 0, 1, 1, 2], [0, 1, 2, 1, 2, 2]],
        data.featr3[:, 1][:, [0, 0, 0, 1, 1, 2], [0, 1, 2, 1, 2, 2]],
        data.featr3[:, 2].reshape(-1, 9)]
        out = torch.hstack((torch.hstack(tmp), data.stmdist))
        
        for i, conv in enumerate(self.convs[:-1]):
            if self.GCNtype == 'MGN':
                out = conv(out, data.edge_index,data.edge_attr) # .unsqueeze
            if self.GCNtype == 'GCN':
                # print('input size:',out.size())
                # print('edge index size:',data.edge_index.size())
                out = conv(out, data.edge_index)
            out = F.relu(out)
        # print('debug8',out.size())
        
        if self.GCNtype == 'MGN':
            out = self.convs[-1](out, data.edge_index,data.edge_attr) # .unsqueeze
        if self.GCNtype == 'GCN':
            out = self.convs[-1](out, data.edge_index) # .unsqueeze
        return out

